In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from sklearn.metrics import accuracy_score # for calculating accuracy of model
from sklearn.model_selection import train_test_split # for splitting the dataset for training and testing
from sklearn.metrics import classification_report # for generating a classification report of model
import pickle # saving and loading trained model
from os import path
import numpy as np
import pandas as pd

In [2]:
from keras.layers import Dense, Input, Dropout # importing dense layer
from keras.models import Sequential #importing Sequential layer
from keras.models import model_from_json # saving and loading trained model
import keras

In [5]:
# My Data load
multi_data = pd.read_csv('/content/drive/MyDrive/multi_data.csv')
multi_data.drop(multi_data.columns[0],axis=1,inplace=True)
le2_classes_ = np.load('/content/drive/MyDrive/le2_classes.npy',allow_pickle=True)

In [15]:
X = multi_data.iloc[:,0:93]  # dataset excluding target attribute (encoded, one-hot-encoded,original)
Y = multi_data[['Dos','Probe','R2L','U2R','normal']] # target attributes

In [17]:
# splitting the dataset 75% for training and 25% testing
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.25, random_state=42)

In [18]:
n_classes = 5
input_dim = X_train.shape[1]

model = Sequential()

model.add(Input(shape=(input_dim, )))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(n_classes, kernel_initializer='normal'))
model.add(Dense(n_classes,activation='softmax'))
#model.add(Activation('softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 128)               12032     
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_6 (Dense)             (None, 256)               33024     
                                                                 
 dropout_4 (Dropout)         (None, 256)               0         
                                                                 
 dense_7 (Dense)             (None, 128)               32896     
                                                                 
 dropout_5 (Dropout)         (None, 128)               0         
                                                                 
 dense_8 (Dense)             (None, 5)                

In [19]:
mlp = model
# defining loss function, optimizer, metrics and then compiling model
mlp.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
# training the model on training dataset
history = mlp.fit(X_train, y_train, epochs=50, batch_size=5000,validation_split=0.2)

Epoch 1/50
16/16 [==============================] - 2s 85ms/step - loss: 1.1180 - accuracy: 0.7942 - val_loss: 0.4776 - val_accuracy: 0.8707
Epoch 2/50
16/16 [==============================] - 1s 70ms/step - loss: 0.3704 - accuracy: 0.8922 - val_loss: 0.3153 - val_accuracy: 0.9258
Epoch 3/50
16/16 [==============================] - 1s 70ms/step - loss: 0.2750 - accuracy: 0.9297 - val_loss: 0.2273 - val_accuracy: 0.9436
Epoch 4/50
16/16 [==============================] - 1s 70ms/step - loss: 0.2081 - accuracy: 0.9429 - val_loss: 0.1779 - val_accuracy: 0.9511
Epoch 5/50
16/16 [==============================] - 1s 71ms/step - loss: 0.1707 - accuracy: 0.9517 - val_loss: 0.1485 - val_accuracy: 0.9572
Epoch 6/50
16/16 [==============================] - 2s 110ms/step - loss: 0.1484 - accuracy: 0.9558 - val_loss: 0.1299 - val_accuracy: 0.9621
Epoch 7/50
16/16 [==============================] - 2s 117ms/step - loss: 0.1338 - accuracy: 0.9601 - val_loss: 0.1198 - val_accuracy: 0.9640
Epoch 8/50


In [21]:
# predicting target attribute on testing dataset
test_results = mlp.evaluate(X_test, y_test, verbose=1)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]*100}%')

985/985 [==============================] - 2s 2ms/step - loss: 0.0664 - accuracy: 0.9782
Test results - Loss: 0.06639823317527771 - Accuracy: 97.8186309337616%


In [22]:
predictions = mlp.predict(X_test)
predictions.shape

985/985 [==============================] - 2s 1ms/step


(31494, 5)

In [23]:
def sample_labels(predictions):
  num = predictions.shape[0]
  pre = np.zeros((num,5),dtype=int)
  for i in range(num):
    idx = np.argmax(predictions[i])
    pre[i, idx] = 1

  return pre

In [24]:
pre = sample_labels(predictions)
print(classification_report(y_test, pre, target_names=le2_classes_))

              precision    recall  f1-score   support

         Dos       0.99      0.98      0.98     11484
       Probe       0.96      0.93      0.94      2947
         R2L       0.83      0.74      0.78       274
         U2R       0.00      0.00      0.00        15
      normal       0.98      0.99      0.98     16774

   micro avg       0.98      0.98      0.98     31494
   macro avg       0.75      0.73      0.74     31494
weighted avg       0.98      0.98      0.98     31494
 samples avg       0.98      0.98      0.98     31494



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
